# HoloViz Survey Results and Response

Authors: Demetris Roumis

Welcome! [HoloViz](https://holoviz.org/) offers a suite of open-source tools for comprehensive data visualization in a web browser, including high-level interfaces for quick construction and precise control to facilitate both exploratory analysis and complex dashboard building.

We recently conducted our first-ever user survey, and thanks to the 130+ respondents, we gained valuable insights into how users interact with HoloViz tools and where our documentation could be improved. Since then, we've been busy implementing your suggestions; we've initiated a documentation structure revamp, provided a path to fully interactive documentation, and taken steps to enhance our community, including hosting documentation sprints, establishing a formal governance structure, joining NumFOCUS, launching a Discord server, and conducting a data app contest.

Our future plans focus on two main areas:

1. **Enhancing Reference Materials:** We plan to create more comprehensive reference materials, including API references, detailed parameter descriptions, and complete docstrings.

2. **Assisting with Package Selection:** We aim to guide new users more effectively in choosing the right HoloViz package, by providing clearer guidance on package selection and unifying the reactive API across HoloViz.

We're excited to share this summary of the survey results, our progress, and future plans. 

## Contents:
- [Select Survey Results](#Select-Survey-Results)
  - [Users and their usage of HoloViz](#Users-and-their-usage-of-HoloViz)
    - [User field and role](#User-field-and-role)
    - [Duration of HoloViz use](#Duration-of-HoloViz-use)
    - [First vs. most used HoloViz library](#First-vs.-most-used-HoloViz-library)
    - [hvPlot vs. HoloViews](#hvPlot-vs.-HoloViews)
    - [HoloViz dev environment](#HoloViz-dev-environment)
    - [Common data and other packages](#Common-data-and-other-packages)
    - [Sharing your work](#Sharing-your-work)
    - [Type hints](#Type-hints)
  - [About HoloViz docs](#About-HoloViz-docs)
    - [Docs rating](#Docs-rating)
    - [Overall docs priorities](#Overall-docs-priorities)
    - [Package-specific docs type priorities](#Package-specific-docs-type-priorities)
    - [Package-specific docs topic priorities](#Package-specific-docs-topic-priorities)
    - [HoloViz tutorial](#HoloViz-tutorial)
- [Actions taken and planned in response to the survey](#Actions-taken-and-planned-in-response-to-the-survey) 
  - [Progress and achievements](#Progress-and-achievements)
    - [Documentation structure revamp](#Documentation-structure-revamp)
    - [In-Browser interactive examples](#In-Browser-interactive-examples)
    - [Community building](#Community-building)
  - [Future plans](#Future-plans)
    - [Enhancing reference materials](#Enhancing-reference-materials)
    - [Assisting with package selection](#Assisting-with-package-selection)
- [Closing](#Closing)

## Select Survey Results 

### Users and their usage of HoloViz

Here are some of the key highlights that we learned about a slice of our user community.

#### User field and role

We asked: **"What field do you work in?"** and **"What title best characterizes your role when using HoloViz tools?"**.

HoloViz tools are clearly used across a wide range of domains, including academia, industry, public, and private sectors. The diversity of applications shows the power and adaptability of HoloViz to support data visualization and analysis needs across many areas of work and study. We hope to further broaden its utility, as cross-pollination of ideas and use cases across fields serves to strengthen our ecosystem and drive open-source innovations.

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Quick note:</p>
        Many of you wrote custom responses for several of the questions. We have reviewed and incorporated them into our summaries and responses, but have either collapsed them into 'other' or omitted them in these summary plots so that the displays don't blow up from a flood of unique categories.
</div>

In [1]:
#ignore
import pandas as pd
import numpy as np
import param
import re
from collections import OrderedDict

import panel as pn; pn.extension()
import hvplot.pandas
import holoviews as hv; hv.extension('bokeh')
from holoviews import opts
from holoviews.core.dimension import Dimension
from holoviews.core.data import Dataset
from holoviews.util.transform import dim
from holoviews.plotting.bokeh.element import ColorbarPlot, LegendPlot
from holoviews.plotting.bokeh.styles import line_properties, fill_properties

import colorcet as cc

# create a categorical colormap for the duration question that 
# has a sequential flavor to it
n_colors = 4
cmap_name = 'bmy'
skip = 256//n_colors
dur_cmap = getattr(cc, cmap_name)[::skip]

brand_cmap = { # brand colors (now unused)
    "HoloViews": '#CC0033', # HoloViews red-orange
    "hvPlot": '#336699', # hvPlot blue
    "Panel": '#009933', # Panel green
    "GeoViews": '#339999', # GeoViews mint green
    "Datashader": '#666699', #Datashader purple
    "Param": '#7f7f7f',   # middle gray
    "Colorcet": '#FF6600', #orange
    "Lumen": '#CC0099', # pink from lumen logo
}

cmap = { # adjusted brand colors
    "HoloViews": '#E34A33', # Adjusted HoloViews red-orange
    "hvPlot": '#3182BD', # Adjusted hvPlot blue
    "Panel": '#31A354', # Adjusted Panel green
    "GeoViews": '#66C2A4', # Adjusted GeoViews mint green
    "Datashader": '#807DBA', # Adjusted Datashader purple
    "Param": '#969696',   # Adjusted middle gray
    "Colorcet": '#FD8D3C', # Adjusted orange
    "Lumen": '#E7298A', # Adjusted pink from lumen logo
}


cat_cmap='glasbey_hv'
max_len = 22 # shorten long answers
fontsize = {'yticks': 10}
default_color = '#808080'
line_width = .1

csv = "https://raw.githubusercontent.com/holoviz-community/survey/main/2022/survey_2022_clean.csv"
results = pd.read_csv(csv)
default_tools=['save', 'tap', 'reset']

# hv.opts.defaults(default_tools=default_tools)
hv.opts.defaults(
    hv.opts.Bars(default_tools=default_tools),
    hv.opts.Text(default_tools=default_tools),
    hv.opts.Sankey(default_tools=default_tools),
    # hv.opts.Overlay(default_tools=default_tools)
)

In [2]:
#ignore
class Pie(Dataset):
    group = param.String(default='Pie')
    kdims = param.List(default=[Dimension('x')], bounds=(0, 2))
    vdims = param.List(default=[Dimension('y')])

class PieChart(ColorbarPlot, LegendPlot):
    inner_annulus = param.Number(default=0.5, bounds=(0, 1))
    _plot_methods = dict(single='annular_wedge')
    style_opts = ['cmap'] + line_properties + fill_properties

    def get_data(self, element, ranges, style):
        vdim = dim(element.vdims[0])
        normed_dim = np.cumsum(vdim/vdim.sum()) * np.pi * 2
        angles = normed_dim.apply(element)
        mapping = dict(x=0, y=0, inner_radius=self.inner_annulus, outer_radius=1, start_angle='start', end_angle='end')
        data = {'start': np.concatenate([[0], angles[:-1]]), 'end': angles}
        self._get_hover_data(data, element)
        return data, mapping, style
        
    def get_extents(self, element, ranges, range_type='combined'):
        for d, rs in ranges.items():
            rs.pop('factors', None)
        return (-1, -1, 1, 1)

hv.Store.register({Pie: PieChart}, 'bokeh')
options = hv.Store.options('bokeh')

options.Pie = hv.Options(
    'plot', show_legend=True, xaxis=None, yaxis=None,
    tools=['hover'], frame_width=300, frame_height=300,
    legend_position='right')


In [3]:
#ignore
question = 'What field do you work in?'
title = 'Field of work'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})
df.dropna(inplace=True)

# shorten long answers
df[title] = df[title].astype(str).apply(lambda x: x if len(x) <= max_len+3 else x[:max_len] + '...')

vc = df[title].value_counts()
one_resp = vc[vc == 1].index
df[title] = df[title].apply(lambda x: "Other" if x in one_resp else x)

df_value_counts = pd.DataFrame(df[title].value_counts()).reset_index()
df_value_counts.columns = [title, 'Count']
df_value_counts['Percentage'] = df_value_counts['Count']/df_value_counts['Count'].sum()
df_value_counts['Percentage'] = df_value_counts['Percentage'].map('{:.2%}'.format)


field_plot = Pie(df_value_counts, title).opts(color=title, cmap=cat_cmap, title=title, default_tools=default_tools)
field_plot

:Pie   [Field of work]   (Count,Percentage)

Additionally, approximately 60% of our respondents are "scientists" specializing in either data, research, or applied fields, while another significant portion comprises "engineers" from various domains such as software and machine learning. Although the number of student responses in this survey was limited, we recognize the importance of actively engaging this demographic and will try to improve their turnout in the future.

In [4]:
#ignore
question = 'What title best characterizes your role when using HoloViz tools?'
title = 'Work role'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})
df.dropna(inplace=True)

# shorten long answers
df[title] = df[title].astype(str).apply(lambda x: x if len(x) <= max_len+3 else x[:max_len] + '...')

vc = df[title].value_counts()
one_resp = vc[vc == 1].index
df[title] = df[title].apply(lambda x: "Other" if x in one_resp else x)

df_value_counts = pd.DataFrame(df[title].value_counts()).reset_index()
df_value_counts.columns = [title, 'Count']
df_value_counts['Percentage'] = df_value_counts['Count']/df_value_counts['Count'].sum()
df_value_counts['Percentage'] = df_value_counts['Percentage'].map('{:.2%}'.format)

role_plot = Pie(df_value_counts, title).opts(color=title, cmap=cat_cmap, title=title, default_tools=default_tools)
role_plot

:Pie   [Work role]   (Count,Percentage)

#### Duration of HoloViz use

We asked: **"For how long have you used HoloViz tools?"**. 

We were hoping to have a balance of responses from both our experienced users and newer users with fresh perspectives. Luckily, respondents were roughly split half and half on whether they have used HoloViz tools for over a year. The range of experience levels provides a valuable mix of feedback on both the cultivated expertise that comes with long-term use, as well as opportunities to improve the user experience for those just beginning their journey with HoloViz.

In [5]:
#ignore
question = 'For how long have you used HoloViz tools?'
title = 'Duration with HoloViz'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})
df.dropna(inplace=True)

tenure_answers = ['Less than 6 months', '6 months - 1 year', '1 - 3 years', 'More than 3 years']
tenure_pretty = ['<6 months', '6-12 months', '1-3 years', '>3 years']
tenure_mapper = OrderedDict(zip(tenure_answers, tenure_pretty))
df[title] = df[title].map(tenure_mapper)

vc = df[title].value_counts()
one_resp = vc[vc == 1].index
df[title] = df[title].apply(lambda x: "Other" if x in one_resp else x)

df_value_counts = pd.DataFrame(df[title].value_counts()).reset_index()
df_value_counts.columns = [title, 'Count']
df_value_counts['Percentage'] = df_value_counts['Count']/df_value_counts['Count'].sum()
df_value_counts['Percentage'] = df_value_counts['Percentage'].map('{:.2%}'.format)

# order the durations based on str dict
df_value_counts[title] = pd.Categorical(df_value_counts[title], categories=tenure_pretty, ordered=True)
df_value_counts = df_value_counts.sort_values(title)

tenure_plot = Pie(df_value_counts, title).opts(color=title, cmap=dur_cmap, title=title, default_tools=default_tools)
tenure_plot

:Pie   [Duration with HoloViz]   (Count,Percentage)

#### First vs. most used HoloViz library
We asked: "**What was the first HoloViz Tool that you used?**" and **"Which specific HoloViz package have you used the most?"**.

Many of you started your HoloViz journey working with HoloViews (one of the original HoloViz packages), but are now Panel aficionados. Panel has been surging in popularity, with people from very different backgrounds now creating cool web apps. We are thinking that Panel and hvPlot are probably the appropriate entry points into HoloViz for new users looking to either do dashboarding or data exploration, respectively (more on this thought [later](#Guide-Package-Selection)).

In [6]:
#ignore
used_first_q = 'What was the first HoloViz Tool that you used?'
used_most_q = 'Which specific HoloViz package have you used the most?'

df = results[[used_first_q, used_most_q]].copy()
df.rename({used_first_q:'First', used_most_q: 'Most'}, axis=1, inplace=True)

# Calculate the counts for each source-target pair
edges_counts = df.groupby(['First', 'Most']).size().reset_index(name='value')

# so hacky.. add whitespace to the first strings to prevent circular dep
edges_counts['First'] = edges_counts['First'].apply(lambda x: f" {x}")

# update cmap dictionary with keys having a whitespace str prefix " "
# so that the sankey works.. otherwise circular dependency
new_cmap = {' ' + key: value for key, value in cmap.items()}
cmap.update(new_cmap)

sankey = hv.Sankey(edges_counts).opts(default_tools=default_tools)

sankey.opts(label_position='outer', cmap=cmap, edge_color=dim('Most').str(), 
           node_color=dim('Most').str(), label_text_font_size=f"{fontsize['yticks']}pt")

# Compute text coordinates
left_label_x = sankey.range('x')[0]-80  # x-coordinate for the left label
right_label_x = sankey.range('x')[1]+80  # x-coordinate for the right label
label_y = sankey.range('y')[1]  # y-coordinate for both labels
label_y += label_y*.08 # Adjust up a bit

# Create the text labels
left_label = hv.Text(left_label_x, label_y-10, "First used").opts(text_font_style='bold')
right_label = hv.Text(right_label_x, label_y-10, "Most used").opts(text_font_style='bold')
Title = hv.Text((sankey.range('x')[1]-sankey.range('x')[0])//2, label_y+15, "HoloViz package use").opts(text_font_style='bold')

sankey_labeled = (sankey * left_label * right_label * Title).opts(width=900, height=400, default_tools=default_tools)
sankey_labeled

:Overlay
   .Sankey.I :Sankey   [First,Most]   (value)
   .Text.I   :Text   [x,y]
   .Text.II  :Text   [x,y]
   .Text.III :Text   [x,y]

#### hvPlot vs. HoloViews

We asked: **"If you have used both hvPlot and HoloViews, which do you prefer for data exploration?"**.

Below, we separate the results into respondents that have either used hvPlot or HoloViews the most, as asked in a prior question. Among users who mostly use hvPlot (left), a significant majority of about 92% expressed a preference for hvPlot for data exploration, indicating a strong correlation between usage and preference. Interestingly, among users who mostly use HoloViews (right), the majority (about 63%) still preferred HoloViews, but the margin was narrower. This could suggest that while users tend to prefer the tool they use most often, hvPlot has a notable appeal even among those who primarily use HoloViews."

In [7]:
#ignore
question = 'If you have used both hvPlot and HoloViews, which do you prefer for data exploration?'
hvPlot_title = 'Preferred library (hvPlot users)'
HoloViews_title = 'Preferred library (HoloViews users)'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: hvPlot_title, df.columns[1]: HoloViews_title})

def package_counts(df, title):
    df_counts = pd.DataFrame(df[title].value_counts())
    df_counts = df_counts.rename(columns={title:'Count'})
    df_counts['Percentage'] = df_counts['Count']/df_counts['Count'].sum()
    df_counts['Percentage'] = df_counts['Percentage'].map('{:.2%}'.format)
    df_counts.index.name = 'Package'
    df_counts = df_counts.reset_index()
    df_counts = df_counts.sort_values('Package')
    return df_counts

df_counts_hvPlot = package_counts(df, hvPlot_title)
df_counts_HoloViews = package_counts(df, HoloViews_title)

holoviews_Pie = Pie(df_counts_HoloViews, 'Package', ['Count', 'Percentage']).opts(color='Package', cmap=cmap, title=HoloViews_title, default_tools=default_tools)
hvplot_Pie = Pie(df_counts_hvPlot, 'Package', ['Count', 'Percentage']).opts(color='Package', cmap=cmap, title=hvPlot_title, default_tools=default_tools)

hv_vs_plot = hvplot_Pie + holoviews_Pie
hv_vs_plot

:Layout
   .Pie.I  :Pie   [Package]   (Count,Percentage)
   .Pie.II :Pie   [Package]   (Count,Percentage)

#### HoloViz dev environment

We asked: **"What notebook environment do you use when working with HoloViz tools?"** and **"Where do you write Python scripts when working with HoloViz tools?"**.

Understanding the environments in which our users operate is crucial for optimizing the HoloViz toolset. So, we sought to identify the most common notebook and Python scripting environments among our user base. Jupyter Lab emerged as the favored notebook environment, used by 66% of respondents, suggesting its capabilities align well with HoloViz's strengths. Meanwhile, over 62% of respondents prefer VS Code for scripting, likely reflecting its robust Python development support.

In [8]:
#ignore
question = "What notebook environment do you use when working with HoloViz tools?"
title = "Notebook for HoloViz"
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

nb_premade_answers = ["Jupyter Notebook", 
                     "Jupyter Lab",
                     "VS Code Notebook", 
                     "Colab Notebook",
                     "I don't use notebooks"]

def count_premade_string_occurrences(df, text_column, string_list):
    count_df = pd.DataFrame()
    df[text_column] = df[text_column].str.lower().str.strip()
    for string in string_list:
        string_simp = string.lower().strip()
        count_df[string] = [df[text_column].str.contains(string_simp).sum()]
    count_df = count_df.transpose()
    count_df.columns = ['Count']
    count_df = count_df.sort_values(by='Count')
    # count_df['Percentage'] = count_df['Count']/len(df) # assumes max one match per response
    # count_df['Percentage'] = count_df['Percentage'].map('{:.2%}'.format)
    count_df.index.name = 'Answer'

    return count_df.reset_index()

df_value_counts = count_premade_string_occurrences(df, title, nb_premade_answers)

nb_holoviz_plot = df_value_counts.hvplot.bar(
    x='Answer', y='Count', height=300, width=800,
    invert=True, title=title, xlabel='', line_width=line_width, fontsize=fontsize, color=default_color)
nb_holoviz_plot

:Bars   [Answer]   (Count)

In [9]:
#ignore
question = "Where do you write Python scripts when working with HoloViz tools?"
title = "Scripting with HoloViz"
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

premade_answers = ["Emacs",
                  "PyCharm",
                  "Spyder",
                  "Sublime",
                  "Vim",
                  "VS Code",
                  "I don't use Python scripts",
                  "Jupyter Lab"]

df_value_counts = count_premade_string_occurrences(df, title, premade_answers)

scripting_holoviz_plot = df_value_counts.hvplot.bar(
    x='Answer', y='Count', hover_cols='Percentage', height=300, width=800,
    invert=True, title=title, xlabel='', line_width=line_width, fontsize=fontsize, color=default_color)
scripting_holoviz_plot

:Bars   [Answer]   (Count)

#### Common data and other packages
We asked: **"What data library/structure do you commonly use with HoloViz?"** and **"What other packages do you use alongside HoloViz tools?"**.

Unsurprisingly, Pandas and NumPy are the most commonly used data libraries with HoloViz, reflecting their foundational role in data science. However, Xarray also shows substantial usage, underscoring its relevance for multi-dimensional array operations. The wide range of other packages used alongside HoloViz, including Matplotlib, Plotly, and scikit-learn, illustrates the versatility of HoloViz tools and their integration within diverse workflows. These insights help us enhance HoloViz's compatibility with popular libraries and tools, optimizing user experience.

In [10]:
#ignore
question = "What data library/structure do you commonly use with HoloViz?"
title = 'Data tools with HoloViz'

df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

def dropna_explode_valuecount_sort(df):
    df = df.dropna()[title].str.split(', ').explode()
    df_value_counts = pd.DataFrame(df.value_counts()).reset_index()
    df_value_counts.columns = [title, 'Count']
    df_value_counts = df_value_counts[df_value_counts['Count']>1]  # remove write-ins
    df_value_counts.sort_values('Count', inplace=True)
    return df_value_counts

df_value_counts = dropna_explode_valuecount_sort(df)

data_tools_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=300, 
                xlabel='', color=default_color, line_width=line_width, fontsize=fontsize, 
                title=title)
data_tools_plot

:Bars   [Data tools with HoloViz]   (Count)

In [11]:
#ignore
question = "What other packages do you use alongside HoloViz tools?"
title = 'Other packages with HoloViz'

df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

df_value_counts = dropna_explode_valuecount_sort(df)

other_tools_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=300, 
                xlabel='', color=default_color, line_width=line_width, fontsize=fontsize, 
                title=title)
other_tools_plot

:Bars   [Other packages with HoloViz]   (Count)

#### Sharing your work

We asked: **"How do you share your HoloViz work with others?"** and **"If you share live, running Python apps, what frameworks and platforms do you use?"**.

Understanding how users disseminate their HoloViz work is key to enhancing its collaborative capabilities. Most users tend to share their entire notebooks, highlighting the notebook's value as a comprehensive record of data analysis that combines code, visualizations, and narrative. Exported HTML and internally hosted apps also emerged as common sharing methods, reflecting the need for static and interactive data presentation formats respectively.

In [12]:
#ignore
question = "How do you share your HoloViz work with others?"
title = 'Share HoloViz work'

df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

df_value_counts = dropna_explode_valuecount_sort(df)

share_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=300, 
                xlabel='', color=default_color, line_width=line_width, fontsize=fontsize, 
                title=title)
share_plot

:Bars   [Share HoloViz work]   (Count)

When it comes to sharing live, running Python apps, Flask emerged as the top choice, likely due to its simplicity and flexibility for web app development. However, a diverse range of other platforms like Amazon Web Services, SSH, and Nginx are also employed, indicating the varied requirements of our users in terms of hosting and deployment. These insights inform our efforts to ensure HoloViz tools are compatible and easy to use across various sharing and deployment platforms.

In [13]:
#ignore
question = "If you share live, running Python apps, what frameworks and platforms do you use?"
title = 'App sharing tools'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

df_value_counts = dropna_explode_valuecount_sort(df)

app_tools_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=300, 
                xlabel='', color=default_color, line_width=line_width, fontsize=fontsize, 
                title=title)
app_tools_plot

:Bars   [App sharing tools]   (Count)

#### Type hints

We asked: **"When using Python packages, do you benefit if a package has used type hints and declarations in their code?"**.

As we transition from understanding how users interact with and disseminate their HoloViz work, we also sought insights into what enhances their experience with Python packages more broadly, especially relating to different forms of documentation. A clear majority, over 70%, affirmed that type hints in a package's code are beneficial. This feature, which aids in understanding the expected input and output types of functions, can increase code readability, assist in debugging, and improve IDE tooling support. The response from our users highlights the value of this coding practice, and as a result, we will are discussing how best to add type hints into our code base.

In [14]:
#ignore
question = 'When using Python packages, do you benefit if a package has used type hints'
title = 'Type hints'
df = results.filter(like = question).copy() # get the col based on partial match
df = df.rename(columns={df.columns[0]: title})

df_value_counts = pd.DataFrame(df.value_counts(), columns=['Count']).reset_index()
df_value_counts['Percentage'] = df_value_counts['Count']/df_value_counts['Count'].sum()
df_value_counts['Percentage'] = df_value_counts['Percentage'].map('{:.2%}'.format)

type_hints_Pie = Pie(df_value_counts, title, ['Count', 'Percentage']).opts(
    color=title, cmap=cat_cmap, title=title, default_tools=default_tools) # set2
type_hints_Pie

:Pie   [Type hints]   (Count,Percentage)

### About HoloViz docs

We asked: **"What development activities would help you most right now? "**.

One of the primary purposes of this first survey was to help prioritize much-needed updates to our documentation, with a particular focus on improving the new user experience. And clearly, you agree that documentation is the highest priority:

In [15]:
#ignore
question = "What development activities would help you most right now?"
title = 'General priorities'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

df_value_counts = dropna_explode_valuecount_sort(df)

dev_priority_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=300, 
            xlabel='', color=default_color, line_width=line_width, fontsize=fontsize, title=title)
dev_priority_plot

:Bars   [General priorities]   (Count)

#### Docs rating

We asked: **"How was your documentation experience when you were a new HoloViz user?"**.

Users rated their initial documentation experience with HoloViz on a scale of 1 to 5. The majority had a neutral (35%) or slightly negative (29%) experience, with a quarter of respondents reporting a positive experience (25%). Only a small percentage found their experience to be excellent or unsatisfactory. These results highlight areas for improvement in our documentation to ensure a smoother onboarding experience for new HoloViz users.

In [16]:
#ignore
question = 'How was your documentation experience when you were a new HoloViz user?'
title = 'Docs rating'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

doc_exp_df_counts = df.value_counts().sort_index()

# response percentages
total_count = doc_exp_df_counts.sum()
response_percentages = (doc_exp_df_counts / total_count) * 100

resp_perc_df = pd.DataFrame(response_percentages, columns=['Percentage'])
resp_perc_df = resp_perc_df.assign(Count=doc_exp_df_counts)
resp_perc_df = resp_perc_df.reset_index()

bars = hv.Bars(resp_perc_df)

labels = {1.0: 'Bad', 5.0: 'Excellent'}
annotations = [hv.Text(x, resp_perc_df.loc[resp_perc_df[title] == x, 'Percentage'].values[0], labels[x], halign='center', valign='bottom')
               for x in labels.keys()]

doc_score_plot = (bars * hv.Overlay(annotations)).opts(
    opts.Bars(width=400, height=300, xlabel='Score', ylabel='Percentage', tools=['hover'], title=title, color=default_color, line_width=line_width),
    opts.Text(text_font_size='10pt')
)
doc_score_plot.opts(default_tools=default_tools)

:Overlay
   .Bars.I  :Bars   [Docs rating]   (Percentage,Count)
   .Text.I  :Text   [x,y]
   .Text.II :Text   [x,y]

#### Overall docs priorities

One of the most important docs questions that we asked was overall **“What potential changes to our documentation do you think would most improve the new user experience?”**. For simplicity, below are the results for the *pre-defined* answers (although there were many write-ins that we are taking action on).

The most favored suggestion, with nearly 59% support, was to improve the reference API material with examples, signifying the need for clear, actionable examples in API documentation. About half of the respondents also sought examples of when and how to switch between different HoloViz tools or a guide on choosing the most appropriate package to work with, indicating a demand for more guidance on using the right tool for a particular task.

In [17]:
#ignore
question = 'What potential changes to our documentation do you think would most improve the new user experience?'
title = 'Docs changes for new users'
df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

premade_answers = ['Create a guide on choosing the most appropriate package to work with',
                   'Create a short tutorial for each individual package',
                   'Create examples of when and how to switch between different packages',
                   'Improve the reference API material with examples and type declarations', 
                   'Redesign the appearance and story of homepages']

premade_counts_df = count_premade_string_occurrences(df, title, premade_answers)

doc_changes = premade_counts_df.sort_values(by='Count').hvplot.bar(
    x='Answer', y='Count', hover_cols='Percentage', height=300, width=800,
    invert=True, title=title, xlabel='', fontsize=fontsize, line_width=line_width, color=default_color)
doc_changes

:Bars   [Answer]   (Count)

#### Package-specific docs type priorities

About each respondent's most used package, we asked **"For [package], rank the type of documentation that we should focus on improving to help you most right now?"**. Below are the results for the three most popular HoloViz packages.

The results indicate diverse needs across these packages. For Panel and hvPlot, 'How-to' recipes for specific tasks emerged as a priority (see [What has been accomplished](#What-has-been-accomplished)). These practical guides can help users navigate specific use-cases and tasks, reinforcing understanding through application. This suggests that users are seeking more actionable guidance on using these packages to address specific challenges or scenarios.

On the other hand, HoloViews users found 'Explanation of concepts and design' to be the most beneficial. This implies that users find the conceptual underpinnings and design principles of HoloViews critical for the effective use of the package. As we revamp our documentation, these user priorities will guide our focus, ensuring we deliver information that is both useful and relevant to our users.

In [18]:
#ignore
packages = ['hvPlot', 'HoloViews', 'Panel']
title='Docs type priority'

all_rankings = pd.DataFrame()

for package in packages:
    question = f"For {package}, rank the type of documentation"
    df = results.filter(like = question).copy()
    df = df.apply(pd.to_numeric, errors='coerce')
    
    df = df.mean()

    # create the yaxis labels from the question ending, e.g. "[Beginner tutorials]"
    df.index = df.index.str.split('[').str[-1].str.split(']').str[0].str.strip()
    
    df = df.reset_index().rename(columns={'index': 'Documentation Type', 0: 'Average Ranking'})
    df['Package'] = package  # Add a column for package name
    
    all_rankings = pd.concat([all_rankings, df])

all_rankings_sorted = all_rankings.sort_values('Average Ranking')

doctype_priority = all_rankings_sorted.hvplot.bar('Documentation Type', 'Average Ranking', by='Package', invert=True, 
                                                  width=800, height=400, xlabel='', fontsize=fontsize, title=title).opts(
                                                  multi_level=False, cmap=cmap, line_width=line_width)
doctype_priority

:Bars   [Documentation Type,Package]   (Average Ranking)

#### Package-specific docs topic priorities

In addition to understanding the **types** of documentation our users find most helpful, we were also interested in identifying specific **topics** within those documentation types where users saw room for improvement. To this end, we again segmented respondents based on their most-used package - Panel, hvPlot, or HoloViews - and asked: **"For the package that you selected, improvement to what documentation topics would you most benefit from?"**.

Panel users expressed a need for better documentation on app responsivity and building custom components. These topics are central to creating and managing effective Panel applications, and users' responses indicate the need for clearer or more comprehensive guidance in these areas.

In [19]:
#ignore

package = 'Panel'
question = f"For {package}, improvement to which specific documentation topics would you most benefit from"
title=f"{package} docs topic priorities"

df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

df_value_counts = dropna_explode_valuecount_sort(df)

panel_priority_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=400, 
                xlabel='', color=cmap[package], line_width=line_width, fontsize=fontsize, title=title)

panel_priority_plot

:Bars   [Panel docs topic priorities]   (Count)

In contrast, hvPlot and HoloViews users were focused on different topics. A clear need for more guidance on interactivity emerged, suggesting users are keen to leverage the interactive capabilities of these packages but may find the current documentation lacking. In addition, users expressed a desire for better integration with other HoloViz packages, underscoring the importance of cohesive, cross-package documentation. The request for improved guidance on applying customizations points to users' desire for more personalized, adaptable visualizations.

In [20]:
#ignore
package = 'hvPlot'
question = f"For {package}, improvement to which documentation topics would you most benefit from"
title=f"{package} docs topic priorities"

df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

df_value_counts = dropna_explode_valuecount_sort(df)

hvPlot_priority_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=400, 
                xlabel='', color=cmap[package], line_width=line_width, fontsize=fontsize, title=title)
hvPlot_priority_plot

:Bars   [hvPlot docs topic priorities]   (Count)

In [21]:
#ignore
package = 'HoloViews'
question = f"For {package}, improvement to which documentation topics would you most benefit from"
title=f"{package} docs topic priorities"

df = results.filter(like = question).copy()
df = df.rename(columns={df.columns[0]: title})

# Split the strings on commas that are not inside parentheses
per_answer = df.dropna()[title].apply(lambda x: re.split(',(?![^()]*\))', x)).explode()

df_value_counts = pd.DataFrame(per_answer.value_counts()).reset_index()
df_value_counts.columns = [title, 'Count']
df_value_counts = df_value_counts[df_value_counts['Count']>1]
df_value_counts.sort_values('Count', inplace=True)

HoloViews_priority_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=400, 
                xlabel='', color=cmap[package], line_width=line_width, fontsize=fontsize, title=title)

HoloViews_priority_plot

:Bars   [HoloViews docs topic priorities]   (Count)

#### HoloViz tutorial

We asked: **"Have you checked out the tutorial on HoloViz.org? If so, in what forms have you experienced it?"**.

When asked about their engagement with the HoloViz tutorial, most users reported reading it directly on the website, signifying the convenience and immediacy of this method. However, many also downloaded the tutorial notebooks for a more hands-on, interactive learning experience. Guided talks through the tutorial were another popular choice, underscoring the importance of providing diverse learning formats.

Despite these varied approaches, fewer users utilized cloud infrastructure for tutorial access, suggesting that this option might need more visibility or user-friendly features. A small fraction were unaware of the tutorial, highlighting an opportunity to improve communication and resource visibility. 

In [22]:
#ignore
question = "Have you checked out the tutorial on HoloViz.org? If so, in what forms have you experience it?"
title = 'Tutorial'

df = results.filter(like=question).copy()
df.columns = [title]

per_answer = df[title].dropna().str.split(', ').explode()
# catch answers about not being aware of the tutorial
not_aware = ["didnt know", "didn't know", "just learned about it", "I have not", "didn't realize", "just looked at it"]
lower_responses = per_answer.str.lower()
not_aware_count = sum(any(phrase in response for phrase in not_aware) for response in lower_responses)

df_value_counts = per_answer.value_counts().rename_axis(title).reset_index(name='Count')
df_value_counts = df_value_counts[df_value_counts['Count']>1]  # remove write-ins

not_aware_df = pd.DataFrame({title: ["write-ins [I was not aware of the tutorial]"], 'Count': [not_aware_count]})
df_value_counts = pd.concat([df_value_counts, not_aware_df], ignore_index=True)

df_value_counts.sort_values('Count', inplace=True)

tutorial_plot = df_value_counts.hvplot.bar(title, invert=True, frame_width=300, height=300, 
                xlabel='', color=default_color, line_width=line_width, fontsize=fontsize, 
                title=title)
tutorial_plot

:Bars   [Tutorial]   (Count)

## Actions taken and planned in response to the survey

### Progress and achievements

#### Documentation structure revamp
A key point of feedback has been the lack of a user-centric structure in our documentation. Historically, our docs have leaned heavily on comprehensive user guides for each topic, making it challenging for users to locate necessary information and for contributors to identify and fill gaps. In response, we've initiated a transition, beginning with Panel's documentation, to adopt a structure inspired by the [Diataxis framework](https://diataxis.fr/). Diataxis prioritizes understanding user needs and organizes documentation around distinct types such as how-to guides, references, and explanations. Our progress so far includes migrating user guides to how-to guides, adding an Explanation section, and overhauling the App Gallery:

- Migrate user guide to how-to guides ([#4244](https://github.com/holoviz/panel/pull/4244), [#4251](https://github.com/holoviz/panel/pull/4251), [#4267](https://github.com/holoviz/panel/pull/4267), [#4290](https://github.com/holoviz/panel/pull/4290), [#4412](https://github.com/holoviz/panel/pull/4412), [#4422](https://github.com/holoviz/panel/pull/4422), [#4759](https://github.com/holoviz/panel/pull/4759), [#4774](https://github.com/holoviz/panel/pull/4774))
- Add Explanation section ([#2797](https://github.com/holoviz/panel/pull/2797), [#3168](https://github.com/holoviz/panel/pull/3168), [#4664](https://github.com/holoviz/panel/pull/4664))
- Overhaul App Gallery ([#4047](https://github.com/holoviz/panel/pull/4047), [#4565](https://github.com/holoviz/panel/pull/4565), [#4574](https://github.com/holoviz/panel/pull/4574), [#4598](https://github.com/holoviz/panel/pull/4598), [#4683](https://github.com/holoviz/panel/pull/4683))

#### In-Browser interactive examples
The survey highlighted a desire for fully interactive documentation. We've taken strides to ensure that most of [Panel's documentation](https://panel.holoviz.org/getting_started/build_app.html) can be interactively executed directly in the web browser. This has already elicited positive feedback, with other software teams showing interest in adopting this approach:

- Use pyodide rendering throughout documentation and add JupyterLite links ([#4751](https://github.com/holoviz/panel/pull/4751))

#### Community building
Aligned with survey feedback for further creation and fostering of the HoloViz community, we've taken tangible steps to enhance our community. Here's what we've accomplished since the survey:

- Hosted sprints at PyData and EuroPython, focusing on documentation.
- Established a formal governance structure for HoloViz, including a code of conduct and a steering committee.
- Obtained fiscal sponsorship from NumFOCUS, further aligning HoloViz with a broader community of open-source projects and opening opportunities for increased collaboration.
- Shifted our user and contributor chats to Discord, enhancing community interaction and transparency.
- Conducted a Panel app contest offering substantial prizes.

### Future plans

#### Enhancing reference materials

Improving the reference material is clearly important to users regardless of experience level. Many of you wrote in additional comments related to this theme. For instance:
- “It would be great if mentions of various functions, etc in the examples were hyperlinked to an API reference (and the API reference had examples).”
- “extensive description of all supported parameters and expected parameter options”
- “…Stories are great, but without API docs, I can’t figure out what Holoviews is actually doing.”
- “[for hvPlot] since there isn't a searchable API reference, it's difficult to figure out how to use them at all or if I should be trying to use them.”
- “...links to reference pages [from other parts of the documentation]”
- “hv.help(<element>) should tell you [what to check in the external plotting library documentation]”
- “...make more complete docstrings. It's usually tough to understand what options are available.”
- “...clear overview of all the possible settings…”
- “Add type hints and stubs to allow static check and autocomplete. That is the most lacking feature of param.”


##### Planned actions for reference materials:
We aim to significantly improve our reference materials by:
- Creating API reference pages for all HoloViz libraries
- Including links from elsewhere in the docs to the reference material
- Enhancing hv.help() with better outputs, sorting, and parameter support:
    - Add output to reference pages (e.g. HoloViews [#5423](https://github.com/holoviz/holoviews/issues/5423))
    - Show the docstring (e.g. HoloViews [#5421](https://github.com/holoviz/holoviews/issues/5421), [#4076](https://github.com/holoviz/holoviews/issues/4076))
    - Sort the output alphabetically (e.g. HoloViews [#5420](https://github.com/holoviz/holoviews/issues/5420))
    - Clarify the distinction of different `tools` args (e.g. HoloViews [#4636](https://github.com/holoviz/holoviews/issues/4636), [#5231](https://github.com/holoviz/holoviews/issues/5231))
    - Ensure the parameters are supported (e.g. HoloViews [#2887](https://github.com/holoviz/holoviews/issues/2887))
- Align and organize reference guides (see proposal on Panel [#4305](https://github.com/holoviz/panel/issues/4305))
- Standardize and complete docstrings:
    - Document input data format (e.g. HoloViews [#3632](https://github.com/holoviz/holoviews/issues/3632), [#2925](https://github.com/holoviz/holoviews/issues/2925), [#2116](https://github.com/holoviz/holoviews/issues/2116))
    - Write consistent docstrings (e.g. HoloViews [#2322](https://github.com/holoviz/holoviews/issues/2322), nbsite [#67](https://github.com/holoviz-dev/nbsite/issues/67)
    - Add type hints to code
- Create a friendly display for Param.Parameterized objects (e.g. Param [#425](https://github.com/holoviz/param/pull/425))
- Fuzzy matching of not-yet-supported backend-specific options (e.g. HoloViews [#4463](https://github.com/holoviz/holoviews/pull/4463)



#### Assisting with package selection

There's notable confusion about choosing the right HoloViz package for a task. Respondents found it difficult to understand the boundaries and overlaps between the packages and to decide which was best for their application:

- “The other issue that I know I share with many of my colleagues is the confusion of the HoloViz package separations. For a beginner it is really hard to grasp where the boundaries are and what the individual package is doing in particular, especially because they can have all different sorts of backends (matplotlib, bokeh, pyplot) and seem to have some overlap (holoviews/geoviews).”
- “in general I found it difficult to easily decide which of the many packages was best for my application”
- “...I always struggle with the many different options of doing something…”
- “The number of subproject (panel, colorcet...) is somewhat confusing.”
- “Hard to learn … when to switch to holoviews from hvplot…”


##### Planned actions for package selection:

- Guide new users to HoloViz.org from our individual library sites to offer a comprehensive view of the ecosystem. Many of our standalone library sites (such as [datashader.org](https://datashader.org/)) do not sufficiently highlight their part in the wider HoloViz ecosystem or suggest alternative packages that could be more suitable. We aim to address this by prominently signposting HoloViz.org on each library homepage, directing new users to a hub where they can receive guidance on selecting the most appropriate package.

- Overhaul the 'Getting Started' section on HoloViz.org to provide clearer guidance on package selection. We understand that our current resources may not adequately guide new users to the most suitable package for their use case. Our plan is to enhance the 'Getting Started' experience by recommending most users to begin with either hvPlot or Panel. These two packages collectively offer access to the majority of HoloViz tools and effectively cater to users either in data exploration mode (hvPlot) or app-building stage (Panel). We also aim to clarify the use-case boundaries between packages with overlapping functionality, like hvPlot and HoloViews, to alleviate confusion about initial package selection and subsequent transitioning between them.

- Unify the reactive API across HoloViz to simplify the creation of UI components and data pipelines. We recognize that the current diversity and inconsistency in our approach to building reactive UI components across different packages can complicate package selection and transition between methods. To address this, we're working on unifying HoloViz's reactive programming approach. This will make it more intuitive and straightforward to construct reactive data pipelines and UI components across the ecosystem, thereby clarifying the appropriate tool selection for specific workflows. Follow the discussion on Holoviz [#370](https://github.com/holoviz/holoviz/issues/370).


## Closing

As an open-source project, HoloViz thrives on contributions from our diverse community of users. Our goal is not just to develop powerful data visualization tools, but also to build a strong, active community of contributors. If any of the future plans resonate with you, we encourage you to get involved. You can reach out in our [#new-contributors channel on Discord](https://discord.gg/dkZhFDNgur) or engage in a relevant issue on [GitHub](https://github.com/holoviz). Many of the improvements we've made so far have been thanks to contributions from both new and existing community members, who we always acknowledge in our release posts - as can be seen on the [Announcements on our Discourse forum](https://discourse.holoviz.org/c/announcements/10). We look forward to your participation in shaping the future of HoloViz.

In closing, we want to express our profound gratitude to all who participated in our first-ever survey. Your time, feedback, and insights are invaluable in guiding our development and refining our focus. Thank you for your continued support and engagement with HoloViz.